In [15]:
from openai import OpenAI

client = OpenAI(api_key="0", base_url="http://192.168.106.26:20000/v1")

def llm_qwen(prompt, model="Qwen3", temperature=0.7, top_p=0.8, max_tokens=1024, presence_penalty=1.5):
    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        presence_penalty=presence_penalty,
    )
    return chat_response.choices[0].message.content

if __name__ == "__main__":
    llm_output = llm_qwen("你好")
    print(llm_output)

你好！有什么我可以帮你的吗？😊


In [ ]:
import json
import re
import torch
import concurrent.futures
from transformers import AutoModel, AutoTokenizer

class AggregationLayer(torch.nn.Module):
    def __init__(self, hidden_size=1024, num_perspectives=3):
        super().__init__()
        self.aggregate_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_perspectives, hidden_size),
            torch.nn.SiLU(),
            torch.nn.Linear(hidden_size, num_perspectives)
        )
        self.softmax = torch.nn.Softmax(dim=-1)
        
    def forward(self, encoded_responses):
        concat_responses = torch.cat(encoded_responses, dim=0).unsqueeze(0)
        weights = self.softmax(self.aggregate_layer(concat_responses))
        weighted_sum = torch.sum(
            torch.stack(encoded_responses) * weights.squeeze(0).unsqueeze(1), 
            dim=0
        )
        return weighted_sum, weights

def load_encoder_model():
    model_name = "/app/sda1/xiangyue/model/chinese-lert-large"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return model, tokenizer

def encode_text(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0)

def process_perspective(key, question, model, tokenizer, llm_qwen):
    response = llm_qwen(question)
    encoded_response = encode_text(response, model, tokenizer)
    return key, response, encoded_response

def generate_perspectives(metaprompt, p):
    return f"""
    对这个问题{metaprompt}加上{p}不同视角的前缀标签，注意不要改变原句子，形成{p}个问题
    只输出格式如下：
    {{
        "one": "",
        "two": "",
        "three": "",
        ...
    }}
    """

def multi_perspective_analysis(metaprompt, p, topk, llm_qwen):
    # 生成并解析视角问题
    perspectives = json.loads(llm_qwen(generate_perspectives(metaprompt, p)))
    print("初始视角问题生成：")
    print(json.dumps(perspectives, ensure_ascii=False, indent=2))

    # 初始化模型
    aggregation_model = AggregationLayer(hidden_size=1024, num_perspectives=p)
    encoder_model, tokenizer = load_encoder_model()

    # 并行处理视角问题
    with concurrent.futures.ThreadPoolExecutor(max_workers=p) as executor:
        futures = [
            executor.submit(process_perspective, key, question, encoder_model, tokenizer, llm_qwen)
            for key, question in perspectives.items()
        ]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    # 提取结果
    responses = [(key, response) for key, response, _ in results]
    encoded_responses = [encoded for _, _, encoded in results]
    response_texts = [response for _, response, _ in results]
    
    # 如果有有效回答，执行聚合
    if encoded_responses:
        aggregated_output, weights = aggregation_model(encoded_responses)
        
        # 打印权重信息
        print("\n聚合权重:")
        for i, (key, _) in enumerate(responses):
            print(f"{key}: {weights[0, i].item():.4f}")
        
        # 获取并打印topk视角
        top_weights, top_indices = torch.topk(weights[0], topk)
        print(f"\nTop {topk} 视角:")
        for i, idx in enumerate(top_indices):
            key = responses[idx][0]
            print(f"{key}: {top_weights[i].item():.4f}")
        
        # 生成最终结果
        final_title = "\n".join(
            re.sub(r'【[^】]+】', '', response_texts[idx]).strip()
            for idx in top_indices
        )
        print("\n最终聚合结果:")
        print(final_title)
        
        return {
            "final_title": final_title,
            "top_perspectives": [(responses[idx][0], top_weights[i].item()) for i, idx in enumerate(top_indices)],
            "all_weights": [(responses[i][0], weights[0, i].item()) for i in range(len(responses))]
        }
    else:
        print("没有有效的回答可供聚合")
        return None

if __name__ == "__main__":
    metaprompt = "你好"
    result = multi_perspective_analysis(metaprompt, p=5, topk=1, llm_qwen=llm_qwen)